<a href="https://colab.research.google.com/github/mandemadhura/crewai_exercises/blob/main/job_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
#set openai API key
#set serper API key
os.environ["OPENAI_MODEL_NAME"] = "gpt-4-turbo"

In [56]:
from crewai_tools import (
    ScrapeWebsiteTool,
    SerperDevTool,
    MDXSearchTool,
    FileReadTool
)

website_scrapper = ScrapeWebsiteTool()
serper_dev_tool = SerperDevTool()
file_reader_tool = FileReadTool(file_path="./resume.md")
semantic_search_tool = MDXSearchTool(file_path="./resume.md")

In [57]:
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do research and analysis on the "
         "job postings to help job applicants.",
    backstory="You are a very good researcher and you are "
              "highly skilled at extracting pinpoint information "
              "such as qualification and other required tech skills "
              "sought by employers, forming the foundation for "
              "effective application tailoring.",
    verbose=True,
    tools=[website_scrapper, serper_dev_tool]
)

In [58]:
profile_highlighter = Agent(
    role="Profile Highlighter",
    goal="Analyze job applicants profile vs job postings "
         "to perfectly identify match and help them standout in the market.",
    verbose=True,
    tools=[file_reader_tool, semantic_search_tool, serper_dev_tool, website_scrapper],
    backstory="You are highly analytical and has ability to dissect and synthesize "
             "information from different sources and craft comprehensice personal "
             "profile that looks extremely professional laying the foundtion for "
             "resume enhancement."
)

In [59]:
resume_strategist = Agent(
    role="Resume Strategist",
    goal="Find all the best ways possible to highlight the resume "
         "in this competative job market.",
    verbose=True,
    tools=[file_reader_tool, website_scrapper, serper_dev_tool, semantic_search_tool],
    backstory="with strategic mind and eye for details, "
              "you excel at refining resumes which best highlight the "
              "most relevant skills and experiences that resonates with "
              "given job requirements."
)

In [60]:
interview_preparer = Agent(
    role="Interview Preprer",
    goal="Create interview questions and talking points "
         "based on job requirements and resume.",
    verbose=True,
    tools=[file_reader_tool, website_scrapper, serper_dev_tool, semantic_search_tool],
    backstory="Your role is crucial in anticipating the dynmics of the interview "
              "You have the ability to formulate questions and talking points. "
              "That way you prepre candidate for a success ensuring they can "
              "confidently address all aspects of the job they are applying for."
)

In [61]:
researcher_task = Task(
    description="Analyze the job posting URL {job_posting_url} to extract "
                "key skills, experiences and qualification requirements. "
                "Use tools to gather the content. Identify and categorize the requirements.",
    expected_output="A structured list of job requirements, key skills, "
                    "qualification and experiences",
    agent=researcher,
    async_execution=True
)

In [62]:
profile_task = Task(
    description="Compile detailed personal as well as professional profile "
                "using the github URL {github_url} and personl profile "
                "{personal_profile}. Utilize tools to extract and "
                "synthesize information from these sources.",
    expected_output="A profile document that highlights skills, qualification "
                    "experiences, communication style, contributions, interests, etc.",
    agent=profile_highlighter,
    async_execution=True

)

In [63]:
resume_strategist_task = Task(
    description="Based on the profile document from profile highlighter "
                "and job posting requirement from previous tasks, tailor "
                "the resume to highlight most relevant areas. "
                "Make sure this is the best resume but don't make up any "
                "information. Update all the areas like skills, experiences, "
                "qualification etc that better match with job posting.",
    expected_output="An updated document that effectively highlights all the "
                    "areas of the resume relevant to the job.",
    output_file="new_resume.md",
    context=[researcher_task, profile_task],
    agent=resume_strategist,
)

In [64]:
interview_preparer_task = Task(
    description="Create potantial interview questions and talking points "
                "based on the tailored resume and job requirements. "
                "Use tools to generate these questions.",
    expected_output="A document with list of questions and discussion points "
                    "which will help candidate for initial interview process",
    output_file="interview_questions.md",
    context=[researcher_task, profile_task, resume_strategist_task],
    agent=interview_preparer,
    async_execution=True

)

In [65]:
job_application_crew = Crew(
    agents=[researcher, profile_highlighter, resume_strategist, interview_preparer],
    tasks=[researcher_task, profile_task, resume_strategist_task, interview_preparer_task],
    verbose=True
)

In [69]:
job_application_input = {
    "job_posting_url": "https://www.linkedin.com/jobs/search-results/?currentJobId=4246343751&eBP=NON_CHARGEABLE_CHANNEL&keywords=python%20europe&refId=3GhlD1bR2fA3WF5EG0BAyA%3D%3D&trackingId=7bRsdABHAbMgbnhR%2B1jh2g%3D%3D",
    "github_url": "<github_url>",
    "personal_profile": "<about_me>"
}

In [ ]:
result = job_application_crew.kickoff(inputs=job_application_input)

In [ ]:
from IPython.display import Markdown
Markdown(result)